### ■ Image Files resize (32x32)

In [ ]:
import os
import cv2 # 이미지나 영상을 처리하는 모듈

path = ["E:\data10\classification\dogs"] 
# E:\data10\classification\dogs 
# E:\data10\classification\hedgedogs

j = 0
for k in path:
    file_list = os.listdir(k)
    file_name = []
    for i in file_list:
        if  i not in ['resized_2.jpg','resized_3.jpg','resized_4.jpg']:
            file_name.append(int( i[:-4] ))
    file_name.sort()
    file_list = [k + '\\' + str(i) + '.jpg' for i in file_name]
    
    for i in file_list:
        img = cv2.imread(i)
        width, height = img.shape[:2]
        resize = cv2.resize(img, (int(32), int(32)), interpolation=cv2.INTER_CUBIC) # 현재 32 x 32로 resize
        cv2.imwrite("d:\\resize\\" + str(j + 1) + '.jpg', resize)
        j += 1

### ■ Create Labels

In [1]:
path = 'E:\data10\classification\\train_label.csv'
file = open( path, 'w')
for  i  in  range(1, 1630):
    file.write( str(1) + '\n' )
for  i  in  range(1, 978):
    file.write( str(0) + '\n') 
file.close()

In [2]:
path = 'E:\data10\classification\\test_label.csv'
file = open( path, 'w')
for  i  in  range(1, 182):
    file.write( str(1) + '\n' )
for  i  in  range(1, 109):
     file.write( str(0) + '\n') 
file.close()

### ■ Create loader3.py 

 ■ image_load  
 ■ next_batch  
 ■ shuffle_batch  
 ■ label_load  

In [ ]:
import  csv
import  os
import  re
import  cv2
import  random
import  numpy  as  np

def  image_load(path):
    file_list = os.listdir(path)
    file_name = []
    for  i  in  file_list:
        a = int(  re.sub('[^0-9]', '', i )  ) # 숫자가 아닌것은 '' 로 처리 
        file_name.append(a) 
    file_name.sort()

    file_res = [] 
    for  j  in   file_name:
        file_res.append('%s\\%d.jpg' %(path,j)  )

    image = []
    for  k  in  file_res:
        img = cv2.imread(k)
        image.append(img)

    return  np.array(image)

def  label_load( path ):
    file = open(path)
    labeldata = csv.reader(file)
    labellist = []
    for  i   in  labeldata:
        labellist.append(i)

    label = np.array(labellist)
    label = label.astype(int)  # 숫자로 변환 
    label = np.eye(2)[label]
    label = label.reshape(-1,2) 
    return  label


def  shuffle_batch( data_list, label ):
    x = np.arange( len( data_list) )
    random.shuffle(x)
    data_list2 = data_list[x]
    label2 = label[x]
    return   data_list2, label2 


def  next_batch( data1, data2, init,  fina ):
    return  data1[ init : fina ],  data2[init : fina] 

In [7]:
import loader3

train_image='E:\\data10\\classification\\train_resize\\'
train_label= 'E:\\data10\\classification\\train_label.csv'
test_image='E:\\data10\\classification\\test_resize\\'
test_label='E:\\data10\\classification\\test_label.csv'

print(loader3.image_load(train_image).shape)
print(loader3.label_load(train_label).shape)
print(loader3.image_load(test_image).shape)
print(loader3.label_load(test_label).shape)

(2606, 32, 32, 3)
(2606, 2)
(289, 32, 32, 3)
(289, 2)


### ■ Train start ( vgg9.ver)

In [9]:
import os, cv2, random

# 필요한 라이브러리 import 하는 코드 
import numpy as np

from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation, MaxPooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K   # 백엔드가  텐서 플로우로 되어있어서 
                                     # 텐서 플로우 명령어 필요할 때 tf 대신에 
                                     # k 를 쓰겠다라는 의미 

import loader3

NUM_CLASSES = 2

train_image='E:\\data10\\classification\\train_resize\\'
train_label= 'E:\\data10\\classification\\train_label.csv'
test_image='E:\\data10\\classification\\test_resize\\'
test_label='E:\\data10\\classification\\test_label.csv'

print(loader3.image_load(train_image).shape)
print(loader3.label_load(train_label).shape)
print(loader3.image_load(test_image).shape)
print(loader3.label_load(test_label).shape)

x_train = loader3.image_load(train_image)
y_train = loader3.label_load(train_label)
x_test = loader3.image_load(test_image)
y_test = loader3.label_load(test_label)


# 모델 만들기 
input_layer = Input((32,32,3))

x = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)
x = MaxPooling2D(pool_size=2,strides=2)(x)


x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = MaxPooling2D(pool_size=2,strides=2)(x)


x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = MaxPooling2D(pool_size=2,strides=2)(x)



# 완전 연결 계층 
x = Flatten()(x)

x = Dense(1024)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Dense(1024)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x)

model = Model(input_layer, output_layer)

# convolution 층 6개, 완전 연결계층 2개 출력층 1개인 9층 신경망 

model.summary()

# 모델 훈련

model_exists = False


if model_exists:
    
    
    model.load_weights('E:\data10\classification\\resultvgg9.h5')
    
    CLASSES = np.array(['개', '고슴도치' ]) 
    preds = model.predict( x_test[1:2] )   
    preds_value = CLASSES[ np.argmax( preds, axis= -1 ) ] 
    actual_value = CLASSES[ np.argmax( y_test [1:2], axis = -1) ] 
    print ( preds_value)
    print ( actual_value)
    
else:
        
    opt = Adam(lr=0.0005)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    model.fit(x_train
              , y_train
              , batch_size=28
              , epochs=50
              , shuffle=True
               , validation_data=(x_test,y_test) )
    
    
    model.save_weights('E:\data10\classification\\resultvgg9.h5')
    
    model.evaluate(x_test, y_test, batch_size=28)
    
    CLASSES = np.array(['개', '고슴도치' ]) 
    preds = model.predict( x_test[1:2] )   
    preds_value = CLASSES[ np.argmax( preds, axis= -1 ) ] 
    actual_value = CLASSES[ np.argmax( y_test[1:2], axis = -1) ] 
    print ( preds_value)
    print ( actual_value)

(2606, 32, 32, 3)
(2606, 2)
(289, 32, 32, 3)
(289, 2)
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 32, 32, 128)       0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
batch

Epoch 19/50
2606/2606 [==============================] - 3s 1ms/step - loss: 0.5775 - accuracy: 0.7045 - val_loss: 0.6097 - val_accuracy: 0.6851
Epoch 20/50
2606/2606 [==============================] - 3s 1ms/step - loss: 0.5532 - accuracy: 0.7360 - val_loss: 0.7859 - val_accuracy: 0.4083
Epoch 21/50
2606/2606 [==============================] - 3s 1ms/step - loss: 0.5209 - accuracy: 0.7594 - val_loss: 0.5893 - val_accuracy: 0.6401
Epoch 22/50
2606/2606 [==============================] - 3s 1ms/step - loss: 0.4935 - accuracy: 0.7675 - val_loss: 0.6595 - val_accuracy: 0.5294
Epoch 23/50
2606/2606 [==============================] - 3s 1ms/step - loss: 0.4702 - accuracy: 0.7847 - val_loss: 0.6679 - val_accuracy: 0.5329
Epoch 24/50
2606/2606 [==============================] - 3s 1ms/step - loss: 0.4638 - accuracy: 0.7817 - val_loss: 1.1955 - val_accuracy: 0.3979
Epoch 25/50
2606/2606 [==============================] - 3s 1ms/step - loss: 0.5525 - accuracy: 0.7233 - val_loss: 0.8104 - val_ac